<a href="https://colab.research.google.com/github/chimera-org/chimera_v2.0/blob/main/notebooks/eegencoder_experiments/01_load_bcic4_2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Clone your GitHub repository
# IMPORTANT: Replace 'YOUR_GITHUB_REPO_URL' with the actual URL of your repository.
# This command will clone the repository into the /content/ directory.
!git clone https://github.com/chimera-org/chimera_v2.0/

print("✅ Repository cloned.")

fatal: destination path 'chimera_v2.0' already exists and is not an empty directory.
✅ Repository cloned.


In [15]:
print("Contents of /content/chimera_v2.0/")
!ls -F /content/chimera_v2.0/

print("\nContents of /content/chimera_v2.0/src/")
!ls -F /content/chimera_v2.0/src/

Contents of /content/chimera_v2.0/
architecture/	    data/	      Makefile	      regulatory/	src/
benchmarks/	    deployment/       models/	      requirements.txt	tests/
clinical/	    docs/	      notebooks/      research/
CODE_OF_CONDUCT.md  GOVERNANCE.md     pyproject.toml  scripts/
config/		    ISSUE_TRACKER.md  quality/	      security/
CONTRIBUTING.md     LICENSE	      README.md       SECURITY.md

Contents of /content/chimera_v2.0/src/
chimera/  eegencoder/


In [16]:
# Setup and Imports
"""
BCI Competition IV 2a - Data Loading Verification
Purpose: Verify dataset integrity before training EEGEncoder
"""

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install minimal dependencies (Colab already has most)
!pip install mne numpy scipy matplotlib tqdm --quiet

# Add src to path
import sys
sys.path.append('/content/chimera_v2.0/src/') # Updated path to cloned repo

import numpy as np
import matplotlib.pyplot as plt
from eegencoder.data_loader import BCIC4_2A_Loader, verify_dataset

print("✅ Environment ready")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Environment ready


In [17]:
# Creating target directory where GDF files are expected to be
import os

extraction_path = "/content/drive/MyDrive/BCI_IV_2a/"
if not os.path.exists(extraction_path):
    os.makedirs(extraction_path)
    print(f"Created directory: {extraction_path}")
else:
    print(f"Directory already exists: {extraction_path}")

Directory already exists: /content/drive/MyDrive/BCI_IV_2a/


In [18]:
# Extracting zip files into the target directory
zip_file_path = "/content/drive/MyDrive/Motor_Imagery_Datasets/OpenBCI/BCI_cIV_2a/BCICIV_2a_gdf.zip"
extraction_path = "/content/drive/MyDrive/BCI_IV_2a/"

# Unzip the file
!unzip -o "{zip_file_path}" -d "{extraction_path}"

print(f"✅ Dataset extracted to {extraction_path}")

Archive:  /content/drive/MyDrive/Motor_Imagery_Datasets/OpenBCI/BCI_cIV_2a/BCICIV_2a_gdf.zip
  inflating: /content/drive/MyDrive/BCI_IV_2a/A01E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A01T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A02E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A02T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A03E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A03T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A04E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A04T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A05E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A05T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A06E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A06T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A07E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A07T.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A08E.gdf  
  inflating: /content/drive/MyDrive/BCI_IV_2a/A08T.g

In [19]:
# Initialize Loader and Verify Subjects
# Create loader instance
loader = BCIC4_2A_Loader(data_path="/content/drive/MyDrive/BCI_IV_2a/")

# Verify first 3 subjects (training data)
print("=== TRAINING DATA VERIFICATION ===")
for subj_id in [1, 2, 3]:
    X, y = verify_dataset(loader, subject_id=subj_id)
    print()


=== TRAINING DATA VERIFICATION ===


RuntimeError: Event time samples were not unique. Consider setting the `event_repeated` parameter."

In [20]:
file_path = '/content/chimera_v2.0/src/eegencoder/data_loader.py'

print(f"--- Contents of {file_path} ---")
with open(file_path, 'r') as f:
    content = f.read()
    print(content)
print("------------------------------------")

print("\nPlease look for the `Epochs(...)` constructor call and check if `event_repeated='merge'` is present in its arguments.")

--- Contents of /content/chimera_v2.0/src/eegencoder/data_loader.py ---
"""
BCI Competition IV 2a Data Loader
Optimized for Raspberry Pi deployment (minimal dependencies)
"""

import numpy as np
import scipy.io as sio
from mne.io import read_raw_gdf
from mne import events_from_annotations, Epochs
import warnings
warnings.filterwarnings('ignore')

class BCIC4_2A_Loader:
    """
    Minimal loader for BCI Competition IV dataset 2a
    Returns NumPy arrays directly (no MNE objects for Pi compatibility)
    """
    
    def __init__(self, data_path="/content/drive/MyDrive/BCI_IV_2a/"):
        """
        Args:
            data_path: Path to folder containing A01T.gdf, A01E.gdf, etc.
        """
        self.data_path = data_path
        self.sfreq = 250  # Sampling frequency (Hz)
        self.n_channels = 22
        self.n_trials = 288
        self.trial_length = 4  # seconds
        
        # Event codes: corrected for BCI IV 2a
        self.event_id = {
            'left_hand': 1,
    